In [8]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import matplotlib.pyplot as plt
import cv2

import numpy as np
import torchvision
from sklearn.metrics import f1_score

import math

In [3]:
class MaskDataset(Dataset):
    def __init__(self, data_path, pre_transform, transform):
        ######################################TODO######################################
        df = pd.read_csv(data_path + 'train_with_label.csv')
        self.X = []
        for x in df['path']:
            self.X.append(pre_transform(Image.open(x)))
        self.y = df['label']
        
        self.classes = ['0 - w/m/<30', '1 - w/m/<60', '2 - w/m/>60', '3 - w/f/<30', '4 - w/f/<60', '5 - w/f/>60', '6 - i/m/<30', '7 - i/m/<60', '8 - i/m/>60', 
                        '9 - i/f/<30', '10 - i/f/<60', '11 - i/f/>60', '12 - n/m/<30', '13 - n/m/<60', '14 - n/m/>60', '15 - n/f/<30', '16 - n/f/<60', '17 - n/f/>60']
        self.transform = transform   
        ################################################################################

    def __len__(self):
        len_dataset = None
        ######################################TODO######################################
        len_dataset = len(self.X)
        ################################################################################
        return len_dataset

    def __getitem__(self, idx):
        X,y = None, None
        ######################################TODO######################################
        X, y = self.transform(self.X[idx]), self.y[idx]
        ################################################################################
        return X, y

In [5]:
dataset = MaskDataset(
    data_path = "",
    pre_transform = transforms.Compose([
        transforms.Resize((512//3,384//3)),
        transforms.CenterCrop((64, 64)),
    ]),
    transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor()
    ])
)
len(dataset)

18900

In [9]:
dataloader_train_mask = DataLoader(
    dataset,
    batch_size = 64,
    shuffle = True,
    num_workers = 2
)

In [10]:
images, labels = next(iter(dataloader_train_mask))

In [11]:
model = torchvision.models.resnet18(pretrained=True)

In [12]:
CLASS_NUM = 18
model.fc = torch.nn.Linear(in_features=512, out_features=CLASS_NUM, bias=True)
torch.nn.init.xavier_uniform_(model.fc.weight)
stdv = 1. / math.sqrt(model.fc.weight.size(1))
model.fc.bias.data.uniform_(-stdv, stdv)

tensor([ 0.0247,  0.0165, -0.0366, -0.0029,  0.0064,  0.0301, -0.0437, -0.0148,
        -0.0078,  0.0339, -0.0150,  0.0021, -0.0434, -0.0065,  0.0301, -0.0076,
        -0.0355,  0.0225])

In [14]:
model.layer1.requires_grad_(False)
model.layer2.requires_grad_(False)

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(128, eps=1e-

In [15]:
for param, weight in model.named_parameters():
    print(f"파라미터 {param:20} 가 gradient 를 tracking 하나요? -> {weight.requires_grad}")

파라미터 conv1.weight         가 gradient 를 tracking 하나요? -> True
파라미터 bn1.weight           가 gradient 를 tracking 하나요? -> True
파라미터 bn1.bias             가 gradient 를 tracking 하나요? -> True
파라미터 layer1.0.conv1.weight 가 gradient 를 tracking 하나요? -> False
파라미터 layer1.0.bn1.weight  가 gradient 를 tracking 하나요? -> False
파라미터 layer1.0.bn1.bias    가 gradient 를 tracking 하나요? -> False
파라미터 layer1.0.conv2.weight 가 gradient 를 tracking 하나요? -> False
파라미터 layer1.0.bn2.weight  가 gradient 를 tracking 하나요? -> False
파라미터 layer1.0.bn2.bias    가 gradient 를 tracking 하나요? -> False
파라미터 layer1.1.conv1.weight 가 gradient 를 tracking 하나요? -> False
파라미터 layer1.1.bn1.weight  가 gradient 를 tracking 하나요? -> False
파라미터 layer1.1.bn1.bias    가 gradient 를 tracking 하나요? -> False
파라미터 layer1.1.conv2.weight 가 gradient 를 tracking 하나요? -> False
파라미터 layer1.1.bn2.weight  가 gradient 를 tracking 하나요? -> False
파라미터 layer1.1.bn2.bias    가 gradient 를 tracking 하나요? -> False
파라미터 layer2.0.conv1.weight 가 gradient 를 tracking 하나요? -> False
파라미터 l

In [16]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 학습 때 GPU 사용여부 결정. Colab에서는 "런타임"->"런타임 유형 변경"에서 "GPU"를 선택할 수 있음

print(f"{device} is using!")

model.to(device) # Resnent 18 네트워크의 Tensor들을 GPU에 올릴지 Memory에 올릴지 결정함

LEARNING_RATE = 0.0001 # 학습 때 사용하는 optimizer의 학습률 옵션 설정
NUM_EPOCH = 10 # 학습 때 mnist train 데이터 셋을 얼마나 많이 학습할지 결정하는 옵션

loss_fn = torch.nn.CrossEntropyLoss() # 분류 학습 때 많이 사용되는 Cross entropy loss를 objective function으로 사용 - https://en.wikipedia.org/wiki/Cross_entropy
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE) # weight 업데이트를 위한 optimizer를 Adam으로 사용함

dataloaders = dataloader_train_mask

cuda:0 is using!


In [20]:
### 학습 코드 시작
from tqdm.notebook import tqdm 
best_test_accuracy = 0.
best_f1 = 0
best_test_loss = 9999.
train_loss =[]
test_loss = []
for epoch in range(NUM_EPOCH):
    n_iter = 0
    running_loss = 0.
    running_acc = 0.
    running_f1 = 0

    model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    for ind, (images, labels) in enumerate(tqdm(dataloaders)):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

        with torch.set_grad_enabled(True): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
            logits = model(images)
            _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
            loss = loss_fn(logits, labels)

            loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
            optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

        running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
        running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
        running_f1 += f1_score(preds.cpu().numpy(), labels.cpu().numpy(), average='macro')
        n_iter += 1

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders.dataset)
    epoch_acc = running_acc / len(dataloaders.dataset)
    epoch_f1 = running_f1 / n_iter

    train_loss.append(epoch_loss)
    print(f"현재 epoch-{epoch}의 데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, 평균 f1: {epoch_f1:.3f}")
    if best_f1 < epoch_f1: # phase가 test일 때, best accuracy 계산
        best_f1 = epoch_f1
        torch.save(model, 'bestmodel.pth')
    if best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
        best_test_accuracy = epoch_acc
    if best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
        best_test_loss = epoch_loss
        
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")


현재 epoch-0의 데이터 셋에서 평균 Loss : 0.264, 평균 Accuracy : 0.904, 평균 f1: 0.850



현재 epoch-1의 데이터 셋에서 평균 Loss : 0.179, 평균 Accuracy : 0.936, 평균 f1: 0.893



현재 epoch-2의 데이터 셋에서 평균 Loss : 0.141, 평균 Accuracy : 0.950, 평균 f1: 0.914



현재 epoch-3의 데이터 셋에서 평균 Loss : 0.105, 평균 Accuracy : 0.964, 평균 f1: 0.938



현재 epoch-4의 데이터 셋에서 평균 Loss : 0.094, 평균 Accuracy : 0.967, 평균 f1: 0.941



현재 epoch-5의 데이터 셋에서 평균 Loss : 0.085, 평균 Accuracy : 0.971, 평균 f1: 0.954



현재 epoch-6의 데이터 셋에서 평균 Loss : 0.071, 평균 Accuracy : 0.977, 평균 f1: 0.958



현재 epoch-7의 데이터 셋에서 평균 Loss : 0.065, 평균 Accuracy : 0.977, 평균 f1: 0.965



현재 epoch-8의 데이터 셋에서 평균 Loss : 0.061, 평균 Accuracy : 0.980, 평균 f1: 0.969



현재 epoch-9의 데이터 셋에서 평균 Loss : 0.051, 평균 Accuracy : 0.982, 평균 f1: 0.972
학습 종료!
최고 accuracy : 0.9819576740264893, 최고 낮은 loss : 0.05141256064255401


In [22]:
test_dir = '/opt/ml/input/data/eval'

In [23]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.img_paths)

In [24]:
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
transform = transforms.Compose([
    transforms.Resize((512//3,384//3)),
    transforms.CenterCrop((64, 64)),
    transforms.ToTensor()
])
dataset = TestDataset(image_paths, transform)
loader = DataLoader(
    dataset,
    shuffle=False
)

# 모델을 정의합니다. (학습한 모델이 있다면 torch.load로 모델을 불러주세요!)
device = torch.device('cuda')
model = torch.load('bestmodel.pth')
#model = MyModel(num_classes=18).to(device)
model.eval()

# 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
all_predictions = []
for images in loader:
    with torch.no_grad():
        images = images.to(device)
        pred = model(images)
        pred = pred.argmax(dim=-1)
        all_predictions.extend(pred.cpu().numpy())
submission['ans'] = all_predictions

# 제출할 파일을 저장합니다.
submission.to_csv(os.path.join(test_dir, 'resnet18_no_pretrain.csv'), index=False)
print('test inference is done!')

test inference is done!
